In [1]:
import numpy as np

import pandas as pd
import sklearn
from sklearn import preprocessing

import bokeh
#from bokeh.charts import BoxPlot
import bokeh.plotting
from bokeh.plotting import ColumnDataSource
from bokeh.models import LabelSet
from bokeh.models import FuncTickFormatter
from bokeh.io import export_svgs

import bokeh.io
bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
#load each run into a separate dataframe
df1 = pd.read_csv('190926_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df2 = pd.read_csv('191007_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df3 = pd.read_csv('191010_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df4 = pd.read_csv('191015_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df5 = pd.read_csv('191020_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df6 = pd.read_csv('191021_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df7 = pd.read_csv('191022_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df8 = pd.read_csv('191025_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df9 = pd.read_csv('191027_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df10 = pd.read_csv('191031_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df11 = pd.read_csv('191106_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df12 = pd.read_csv('191115_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df13 = pd.read_csv('191119_Deuterium_Transfer_Peak_Areas.csv', comment='#')

In [3]:
#add a column to each dataframe listing the date
df1['date'] = '190926'
df2['date'] = '191007'
df3['date'] = '191010'
df4['date'] = '191015'
df5['date'] = '191020'
df6['date'] = '191021'
df7['date'] = '191022'
df8['date'] = '191025'
df9['date'] = '191027'
df10['date'] = '191031'
df11['date'] = '191106'
df12['date'] = '191115'
df13['date'] = '191119'

#merge data frames into one dataframe
df=df1.append(df2).append(df3).append(df4).append(df5).append(df6).append(df7).append(df8).append(df9).append(df10).append(df11).append(df12).append(df13)

df.tail()

,Species,WellNumber,Peak ID,Ret Time,Start Tm,End Tm,m/z,Area,Area Percent,Height,Height Percent,A/H,Type,Outlier,date
43,L,8,D50C24,9.480,9.475,9.525,TIC,1020,0.01,2002,0.02,0.51,U,N,191119
44,L,8,D62C30,12.380,12.335,12.380,TIC,34904,0.30,33043,0.34,1.06,U,N,191119
45,D,8,C18,7.439,7.405,7.500,TIC,11443903,99.74,9614537,99.71,1.19,U,N,191119
46,D,8,D50C24,9.525,9.475,9.525,TIC,3225,0.03,3986,0.04,0.81,U,N,191119
47,D,8,D62C30,12.380,12.335,12.380,TIC,26344,0.23,24262,0.25,1.09,U,N,191119


In [4]:
#open metdata, which contains information regarding interaction time 
#between ant and beetle
metadata = pd.read_csv('Movement_metadata.csv')

#Date sync contains the dates that the video and GCMS data were collected on
#These two dates are different for every trial
date_sync = pd.read_csv('DCHC_Date_sync.csv')
metadata.head(2)

,Unnamed: 0,well number,species,collection date,file path,length lower,length upper,touching percent
0,0,3,S,191025,./191025/191025_DCHC_transfer_experiment_S_wel...,NaN,110,0.777064
1,1,7,S,191025,./191025/191025_DCHC_transfer_experiment_S_wel...,NaN,110,0.809745


In [5]:
date_sync.head(2)

,Video Date,GCMS Date
0,190919,190926
1,191003,191007


In [6]:
#rename the date column in the metadata and main dataframe so that they
#can be merged with the date_sync dataframe and subsequently merged with
#eachother
metadata = metadata.rename(columns={'collection date':'Video Date','well number':'WellNumber'})
df = df.rename(columns={'date':'GCMS Date'})
len(df)

624

In [7]:
df.head()

,Species,WellNumber,Peak ID,Ret Time,Start Tm,End Tm,m/z,Area,Area Percent,Height,Height Percent,A/H,Type,Outlier,GCMS Date
0,L,1,C18,7.435,7.405,7.470,TIC,5589973,93.17,5956210,95.09,0.94,L,N,190926
1,L,1,D50C24,9.498,9.475,9.525,TIC,231950,3.87,222378,3.55,1.04,L,N,190926
2,L,1,D62C30,12.350,12.330,12.375,TIC,177959,2.97,85013,1.36,2.09,L,N,190926
3,S,1,C18,7.434,7.410,7.455,TIC,6037087,99.11,6123349,99.37,0.99,L,N,190926
4,S,1,D50C24,9.497,9.475,9.525,TIC,37683,0.62,31671,0.51,1.19,L,N,190926


In [8]:
df['GCMS Date']=df['GCMS Date'].astype(int)
df['WellNumber']=df['WellNumber'].astype(int)

In [9]:
df = df.merge(date_sync).merge(metadata[['WellNumber','Video Date','touching percent']])
df.head()

,Species,WellNumber,Peak ID,Ret Time,Start Tm,End Tm,m/z,Area,Area Percent,Height,Height Percent,A/H,Type,Outlier,GCMS Date,Video Date,touching percent
0,L,1,C18,7.435,7.405,7.470,TIC,5589973,93.17,5956210,95.09,0.94,L,N,190926,190919,0.597702
1,L,1,D50C24,9.498,9.475,9.525,TIC,231950,3.87,222378,3.55,1.04,L,N,190926,190919,0.597702
2,L,1,D62C30,12.350,12.330,12.375,TIC,177959,2.97,85013,1.36,2.09,L,N,190926,190919,0.597702
3,S,1,C18,7.434,7.410,7.455,TIC,6037087,99.11,6123349,99.37,0.99,L,N,190926,190919,0.597702
4,S,1,D50C24,9.497,9.475,9.525,TIC,37683,0.62,31671,0.51,1.19,L,N,190926,190919,0.597702


In [10]:
len(df)

606

In [11]:
#function to normalize the peak are of each unique peak using the area of the C18 peak for each sample
def normalize_area(data):
    #check to make sure that the dataframe hasn't already been normalized
    if not {'Normalized Area', 'Hydrocarbon amount'}.issubset(data.columns):
        #Create array of c18 values repeated in groups of three, matching the number of unique peaks for each sample
        c18_Areas = np.repeat(data['Area'].loc[data['Peak ID'] == 'C18'],3).reset_index().drop("index", axis=1)
        
        #Divide the area of each peak in the dataframe by the corresponding c18 peak area
        data['Normalized Area'] = np.divide(data['Area'],c18_Areas['Area'])
        
        #Calculate the hydrocarbon amount, in ng, by multiplying the normalized peak area by 25.
        #25 ng of c18 was injected in each sample (1 microliter of a 25 ng/microliter solution of c18 in hexane)
        data['Hydrocarbon amount'] = data['Normalized Area']*25
        
    return data

In [12]:
df = normalize_area(df)

In [13]:
#Only scale the beetle values by the interaction percent. The ant, in interacting
#with itself has an effective interaction percent of 100
df['Scaled hydrocarbon amount'] = df['Hydrocarbon amount']
df['Scaled hydrocarbon amount'].loc[df['Species'] != 'L'] = df['Hydrocarbon amount'].loc[df['Species'] != 'L'] / df['touching percent'].loc[df['Species'] != 'L']
len(df)

/Users/TomNaragon/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


606

In [14]:
#function to create an identifier for each sample in the order (Liometopum or not : Beetle: Type)
def add_identifier(data):
    #check if identifier column already exists
    if not {'Identifier'}.issubset(data.columns):
        
        #create identifier column, populating it with the beetle species used in each well
        data = data.merge(data[['WellNumber','Species','GCMS Date']].loc[data['Species'] != 'L'].rename(columns={"Species": "Identifier"}).drop_duplicates())
        
        #modify the identifier column, adding a leading 'L' for Liometopum runs and adding a suffix from the 'Type' column
        data['Identifier'] = ((data['Species']=='L')*pd.Series('L',index=data.index)) + data['Identifier'] + data['Type']
        
    return data

In [15]:
df = add_identifier(df)

In [16]:
len(df)

603

In [17]:
sum(df['Outlier']=='Y')

27

In [18]:
df.loc[df['Outlier']=='AI']

,Species,WellNumber,Peak ID,Ret Time,Start Tm,End Tm,m/z,Area,Area Percent,Height,...,A/H,Type,Outlier,GCMS Date,Video Date,touching percent,Normalized Area,Hydrocarbon amount,Scaled hydrocarbon amount,Identifier
30,L,6,C18,7.433,7.405,7.455,TIC,5721617,93.43,6132080,...,0.93,L,AI,190926,190919,0.236348,1.000000,25.000000,25.000000,LSL
31,L,6,D50C24,9.496,9.475,9.525,TIC,266166,4.35,271507,...,0.98,L,AI,190926,190919,0.236348,0.046519,1.162984,1.162984,LSL
32,L,6,D62C30,12.349,12.330,12.375,TIC,136026,2.22,68946,...,1.97,L,AI,190926,190919,0.236348,0.023774,0.594351,0.594351,LSL
33,S,6,C18,7.434,7.405,7.465,TIC,5833603,99.26,6066766,...,0.96,L,AI,190926,190919,0.236348,1.000000,25.000000,105.776037,SL
34,S,6,D50C24,9.497,9.475,9.525,TIC,28105,0.48,25174,...,1.12,L,AI,190926,190919,0.236348,0.004818,0.120444,0.509605,SL
35,S,6,D62C30,12.348,12.330,12.375,TIC,15270,0.26,7190,...,2.12,L,AI,190926,190919,0.236348,0.002618,0.065440,0.276879,SL
66,L,5,C18,7.438,7.405,7.460,TIC,5893710,99.13,6014575,...,0.98,U,AI,191007,191003,0.061301,1.000000,25.000000,25.000000,LPU
67,L,5,D50C24,9.530,9.480,9.530,TIC,1834,0.03,1124,...,1.63,U,AI,191007,191003,0.061301,0.000311,0.007779,0.007779,LPU
68,L,5,D62C30,12.385,12.340,12.385,TIC,50158,0.84,46674,...,1.07,U,AI,191007,191003,0.061301,0.008510,0.212761,0.212761,LPU
69,P,5,C18,7.438,7.405,7.460,TIC,5949861,99.38,5887266,...,1.01,U,AI,191007,191003,0.061301,1.000000,25.000000,407.821848,PU


In [19]:
df = df.loc[df['Outlier']!='Y']
# df = df.loc[df['Outlier']!='AI']
#df = df.loc[df['Outlier'] == 'N']
len(df)

576

In [20]:
np.unique(df['Outlier'])

array(['A', 'AI', 'B', 'BI', 'N'], dtype=object)

In [21]:
def box_and_whisker(data,plt,x_vals,y_vals):
    
    groups = data.groupby(x_vals)
    q1 = groups.quantile(q=0.25)
    q2 = groups.quantile(q=0.5)
    q3 = groups.quantile(q=0.75)
    iqr = q3 - q1
    upper = q3 + 1.5*iqr
    lower = q1 - 1.5*iqr

    # find the outliers for each category
    def outliers(group):
        cat = group.name
        return group[(group[y_vals] > upper.loc[cat][y_vals]) | (group[y_vals] < lower.loc[cat][y_vals])][y_vals]
    out = groups.apply(outliers).dropna()

    # prepare outlier data for plotting, we need coordinates for every outlier.
    if not out.empty:
        outx = []
        outy = []
        for keys in out.index:
            outx.append(keys[0])
            outy.append(out.loc[keys[0]].loc[keys[1]])

    # if no outliers, shrink lengths of stems to be no longer than the minimums or maximums
    qmin = groups.quantile(q=0.00)
    qmax = groups.quantile(q=1.00)
    upper[y_vals] = [min([x,y]) for (x,y) in zip(list(qmax.loc[:,y_vals]),upper[y_vals])]
    lower[y_vals] = [max([x,y]) for (x,y) in zip(list(qmin.loc[:,y_vals]),lower[y_vals])]

    # stems
    plt.segment(q3.reset_index()[x_vals], upper[y_vals], q3.reset_index()[x_vals], q3[y_vals], line_color="black")
    plt.segment(q3.reset_index()[x_vals], lower[y_vals], q3.reset_index()[x_vals], q1[y_vals], line_color="black")

    #boxes
    plt.vbar(q3.reset_index()[x_vals], 0.7, q2[y_vals], q3[y_vals], fill_color=None, line_color="black")
    plt.vbar(q3.reset_index()[x_vals], 0.7, q1[y_vals], q2[y_vals], fill_color=None, line_color="black")

    # whiskers (almost-0 height rects simpler than segments)
    plt.vbar(q3.reset_index()[x_vals], top=lower[y_vals],bottom=lower[y_vals], width=0.2, line_color="black")
    plt.vbar(q3.reset_index()[x_vals], top=upper[y_vals],bottom=upper[y_vals], width=0.2, line_color="black")
    
    return plt

In [22]:
groups = df.groupby('Identifier')
mean_vals = groups.quantile(q=0.5)

In [23]:
mean_vals['Background Subtracted'] = 0
for i in range(int(len(mean_vals)/2)):
    mean_vals['Background Subtracted'].iloc[i*2] = mean_vals['Hydrocarbon amount'].iloc[i*2]-mean_vals['Hydrocarbon amount'].iloc[i*2 + 1]

In [24]:
mean_vals = mean_vals.reset_index()
mean_vals

0.5,Identifier,WellNumber,Ret Time,Start Tm,End Tm,Area,Area Percent,Height,Height Percent,A/H,GCMS Date,Video Date,touching percent,Normalized Area,Hydrocarbon amount,Scaled hydrocarbon amount,Background Subtracted
0,DL,6.0,9.5040,9.475,9.525,58554.0,0.570,44789.0,0.460,1.200,191106.0,191105.0,0.054708,0.005717,0.142928,4.442832,0.081976
1,DU,3.0,9.5250,9.475,9.525,16598.0,0.240,15860.0,0.190,1.090,191106.0,191105.0,0.026790,0.002438,0.060952,6.196680,0.000000
2,LDL,6.0,9.4930,9.475,9.525,1826319.5,12.840,1270023.5,10.805,1.030,191106.0,191105.0,0.054708,0.172792,4.319805,4.319805,4.272101
3,LDU,3.0,9.4990,9.475,9.525,10974.0,0.190,11039.0,0.155,1.085,191106.0,191105.0,0.026790,0.001908,0.047705,0.047705,0.000000
4,LPL,5.0,9.4970,9.475,9.525,560293.0,9.980,488003.0,8.910,1.000,191022.0,191021.0,0.168874,0.136501,3.412534,3.412534,3.342207
5,LPU,5.0,9.5250,9.475,9.525,10983.0,0.280,12095.5,0.250,0.990,191022.0,191021.0,0.124894,0.002813,0.070327,0.070327,0.000000
6,LSL,4.0,9.4975,9.475,9.525,123030.0,2.190,88190.5,1.655,1.010,191020.0,191019.0,0.412548,0.023405,0.585135,0.585135,0.532974
7,LSU,3.0,9.5250,9.475,9.525,9858.0,0.210,10423.0,0.190,0.970,191020.0,191019.0,0.430219,0.002086,0.052161,0.052161,0.000000
8,PL,5.5,9.5025,9.475,9.525,24767.5,0.525,21885.5,0.400,1.015,191023.5,191021.5,0.172314,0.005330,0.133242,0.969432,0.073699
9,PU,5.0,9.5250,9.475,9.525,6888.0,0.240,8914.0,0.280,0.920,191022.0,191021.0,0.124894,0.002382,0.059543,0.540147,0.000000


In [25]:
df_C24 = df.loc[df['Peak ID'] == 'D50C24']
df_C30 = df.loc[df['Peak ID'] == 'D62C30']

In [26]:
plotting_df=df_C24

Identifiers = ['LSU','SU','LSL','SL','LPU','PU','LPL','PL','LDU','DU','LDL','DL',]
Labels={'LSU': '','SU': 'Sceptobius Control','LSL': '','SL': 'Sceptobius Treated',
        'LPU': '','PU': 'Platyusa Control','LPL': '','PL': 'Platyusa Treated',
        'LDU': '','DU': 'Dalotia Control','LDL': '','DL': 'Dalotia Treated'}

np.random.seed(666)  

p = bokeh.plotting.figure(plot_width=800,
                          plot_height=600,
                          title='C24',
                          x_range=Identifiers,
                          y_axis_label='ng D50C24',
                          y_axis_type='log')

colors=['#494949','#5F56FF','#494949','#5F56FF','#494949','#C42F2F','#494949','#C42F2F','#494949','#832161','#494949','#832161']

p = box_and_whisker(plotting_df,p,'Identifier','Hydrocarbon amount')

for index, row in plotting_df[['WellNumber','GCMS Date']].drop_duplicates().iterrows():
        
    data = {'identifier': plotting_df['Identifier'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['GCMS Date']==row['GCMS Date'])].values,
            'hydrocarbon amount': plotting_df['Hydrocarbon amount'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['GCMS Date']==row['GCMS Date'])].values}

    source = ColumnDataSource(data=data)
    outlier=np.unique(plotting_df['Outlier'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['GCMS Date']==row['GCMS Date'])])

    offsetVal=(np.random.rand(1)[0]-0.5)*0.5
    p.line(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
           'hydrocarbon amount',
           source=source,
           color='black',
           alpha=0.3)

    p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
           'hydrocarbon amount',
           source=source,
           color=bokeh.transform.factor_cmap('identifier',colors,Identifiers),
           alpha=0.6,
           size=7)
    
    if outlier=='AI':
        p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
               'hydrocarbon amount',
               source=source,
               color='lime',
               alpha=1,
               size=7,
               legend='Ant Injured')
    if outlier=='A':
        p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
               'hydrocarbon amount',
               source=source,
               color='purple',
               alpha=1,
               size=7,
               legend='Ant Chomped')
    if outlier=='BI':
        p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
               'hydrocarbon amount',
               source=source,
               color='orange',
               alpha=1,
               size=7,
               legend='Beetle Injured')
    if outlier=='B':
        p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
               'hydrocarbon amount',
               source=source,
               color='pink',
               alpha=1,
               size=7,
               legend='Beetle Chomped')
    if outlier=='Y':
        p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
               'hydrocarbon amount',
               source=source,
               color='cyan',
               alpha=1,
               size=7,
               legend='Ant Crushed or Not Labeled')

p.xgrid.visible = False
p.ygrid.visible = False

# Add custom axis

p.xaxis.formatter = FuncTickFormatter(code="""
var labels = %s;
return labels[tick];
""" %Labels)

bokeh.io.show(p)

In [27]:
#Calculating the ratio of the hydrocarbon measured on the beetle to that measured on the ant, for both the normal hydrocarbon amount
#as well as the scaled hydrocarbon amount
df_C24['ant ratio'] = 0
df_C24['ant relative amount'] = 0
df_C24['scaled ant ratio'] = 0
for index, row in df_C24[['WellNumber','GCMS Date','Outlier']].drop_duplicates().iterrows():
   
    if row['Outlier'] == 'Y':
        continue
    
    hydrocarbon_amount = df_C24['Hydrocarbon amount'].loc[(df_C24['WellNumber']==row['WellNumber']) & (df_C24['GCMS Date']==row['GCMS Date'])].values
    scaled_hydrocarbon_amount = df_C24['Scaled hydrocarbon amount'].loc[(df_C24['WellNumber']==row['WellNumber']) & (df_C24['GCMS Date']==row['GCMS Date'])].values
    
    df_C24['ant ratio'].loc[(df_C24['WellNumber']==row['WellNumber']) & (df_C24['GCMS Date']==row['GCMS Date']) & (df_C24['Species']=='L')] = [hydrocarbon_amount[0]/hydrocarbon_amount[0]]
    df_C24['ant ratio'].loc[(df_C24['WellNumber']==row['WellNumber']) & (df_C24['GCMS Date']==row['GCMS Date']) & (df_C24['Species']!='L')] = [hydrocarbon_amount[1]/hydrocarbon_amount[0]]
    
    df_C24['ant relative amount'].loc[(df_C24['WellNumber']==row['WellNumber']) & (df_C24['GCMS Date']==row['GCMS Date']) & (df_C24['Species']=='L')] = [0]
    df_C24['ant relative amount'].loc[(df_C24['WellNumber']==row['WellNumber']) & (df_C24['GCMS Date']==row['GCMS Date']) & (df_C24['Species']!='L')] = [(hydrocarbon_amount[1]-hydrocarbon_amount[0])/hydrocarbon_amount[0]]

    df_C24['scaled ant ratio'].loc[(df_C24['WellNumber']==row['WellNumber']) & (df_C24['GCMS Date']==row['GCMS Date']) & (df_C24['Species']=='L')] = [scaled_hydrocarbon_amount[0]/scaled_hydrocarbon_amount[0]]
    df_C24['scaled ant ratio'].loc[(df_C24['WellNumber']==row['WellNumber']) & (df_C24['GCMS Date']==row['GCMS Date']) & (df_C24['Species']!='L')] = [scaled_hydrocarbon_amount[1]/scaled_hydrocarbon_amount[0]]

    

df_C24['ant ratio']=df_C24['ant ratio'].astype('float64')
df_C24['scaled ant ratio']=df_C24['scaled ant ratio'].astype('float64')

/Users/TomNaragon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/TomNaragon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/TomNaragon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

In [28]:
plotting_df=df_C24.loc[df_C24['Species']!='L']

#Identifiers = ['SU','SL','PU','PL','DU','DL',]
Identifiers = ['SL','PL','DL',]
Labels={'LSU': '','SU': 'Sceptobius Control','LSL': '','SL': 'Sceptobius',
        'LPU': '','PU': 'Platyusa Control','LPL': '','PL': 'Platyusa',
        'LDU': '','DU': 'Dalotia Control','LDL': '','DL': 'Dalotia'}

np.random.seed(666)  

p = bokeh.plotting.figure(plot_width=800,
                          plot_height=500,
                          x_range=Identifiers,
                          y_axis_label='Deuterated hydrocarbon amount relative to ant',
                          y_axis_type='linear')

colors=['#5F56FF','#C42F2F','#832161']


p = box_and_whisker(plotting_df,p,'Identifier','ant relative amount')

for index, row in plotting_df[['WellNumber','GCMS Date']].drop_duplicates().iterrows():

    outlier=np.unique(plotting_df['Outlier'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['GCMS Date']==row['GCMS Date'])])
        

    data = {'identifier': plotting_df['Identifier'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['GCMS Date']==row['GCMS Date'])].values,
            'hydrocarbon amount': plotting_df['ant relative amount'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['GCMS Date']==row['GCMS Date'])].values}

    source = ColumnDataSource(data=data)    
    
    offsetVal=(np.random.rand(1)[0]-0.5)*0.3
    
    p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
           'hydrocarbon amount',
           source=source,
           color=bokeh.transform.factor_cmap('identifier',colors,Identifiers),
           alpha=0.7,
           size=7)

#     if outlier=='A':
#         p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
#                'hydrocarbon amount',
#                source=source,
#                color=bokeh.transform.factor_cmap('identifier',colors,Identifiers),
#                alpha=1,
#                size=7)
        
#     if outlier=='B':
#         p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
#                'hydrocarbon amount',
#                source=source,
#                color=bokeh.transform.factor_cmap('identifier',colors,Identifiers),
#                alpha=1,
#                size=7)
    
p.xgrid.visible = False
p.ygrid.visible = False

# Add custom axis

p.xaxis.formatter = FuncTickFormatter(code="""
var labels = %s;
return labels[tick];
""" %Labels)


p.xaxis.axis_label_text_font_size='15pt'
p.yaxis.axis_label_text_font_size='15pt'
p.yaxis.axis_label_text_font_style='bold'
p.xaxis.axis_label_text_font_style = "italic"

p.xaxis.major_label_text_font_size = "12pt"
p.yaxis.major_label_text_font_size = "14pt"

bokeh.io.show(p)

In [38]:
plotting_df=df_C24.loc[df_C24['Species']!='L']

#Identifiers = ['SU','SL','PU','PL','DU','DL',]
Identifiers = ['SL','PL','DL',]
Labels={'LSU': '','SU': 'Sceptobius Control','LSL': '','SL': 'Sceptobius',
        'LPU': '','PU': 'Platyusa Control','LPL': '','PL': 'Platyusa',
        'LDU': '','DU': 'Dalotia Control','LDL': '','DL': 'Dalotia'}

np.random.seed(666)  

p = bokeh.plotting.figure(plot_width=800,
                          plot_height=500,
                          x_range=Identifiers,
                          y_axis_label='Deuterated hydrocarbon amount relative to ant',
                          y_axis_type='linear')

colors=['#5F56FF','#C42F2F','#832161']


p = box_and_whisker(plotting_df,p,'Identifier','ant ratio')

for index, row in plotting_df[['WellNumber','GCMS Date']].drop_duplicates().iterrows():

    outlier=np.unique(plotting_df['Outlier'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['GCMS Date']==row['GCMS Date'])])
        

    data = {'identifier': plotting_df['Identifier'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['GCMS Date']==row['GCMS Date'])].values,
            'hydrocarbon amount': plotting_df['ant ratio'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['GCMS Date']==row['GCMS Date'])].values}

    source = ColumnDataSource(data=data)    
    
    offsetVal=(np.random.rand(1)[0]-0.5)*0.3
    
    p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
           'hydrocarbon amount',
           source=source,
           color=bokeh.transform.factor_cmap('identifier',colors,Identifiers),
           alpha=0.7,
           size=7)

#     if outlier=='A':
#         p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
#                'hydrocarbon amount',
#                source=source,
#                color=bokeh.transform.factor_cmap('identifier',colors,Identifiers),
#                alpha=1,
#                size=7)
        
#     if outlier=='B':
#         p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
#                'hydrocarbon amount',
#                source=source,
#                color=bokeh.transform.factor_cmap('identifier',colors,Identifiers),
#                alpha=1,
#                size=7)
    
p.xgrid.visible = False
p.ygrid.visible = False

# Add custom axis

p.xaxis.formatter = FuncTickFormatter(code="""
var labels = %s;
return labels[tick];
""" %Labels)


p.xaxis.axis_label_text_font_size='15pt'
p.yaxis.axis_label_text_font_size='15pt'
p.yaxis.axis_label_text_font_style='bold'
p.xaxis.axis_label_text_font_style = "italic"

p.xaxis.major_label_text_font_size = "12pt"
p.yaxis.major_label_text_font_size = "14pt"

bokeh.io.show(p)

In [29]:
plotting_df.head()

,Species,WellNumber,Peak ID,Ret Time,Start Tm,End Tm,m/z,Area,Area Percent,Height,...,GCMS Date,Video Date,touching percent,Normalized Area,Hydrocarbon amount,Scaled hydrocarbon amount,Identifier,ant ratio,ant relative amount,scaled ant ratio
4,S,1,D50C24,9.497,9.475,9.525,TIC,37683,0.62,31671,...,190926,190919,0.597702,0.006242,0.156048,0.261080,SL,0.150430,-0.849570,0.251680
10,S,2,D50C24,9.525,9.475,9.525,TIC,2344,0.04,1432,...,190926,190919,0.235601,0.000388,0.009708,0.041206,SU,1.545441,0.545441,6.559562
16,S,3,D50C24,9.525,9.475,9.525,TIC,1573,0.03,829,...,190926,190919,0.282222,0.000257,0.006416,0.022733,SU,0.336422,-0.663578,1.192050
22,S,4,D50C24,9.497,9.475,9.525,TIC,18172,0.30,15051,...,190926,190919,0.301394,0.003003,0.075063,0.249053,SL,0.113792,-0.886208,0.377551
28,S,5,D50C24,9.525,9.475,9.525,TIC,2132,0.04,1181,...,190926,190919,0.306458,0.000356,0.008899,0.029037,SU,0.388009,-0.611991,1.266110


In [30]:
hydrocarbon_amount[1]/hydrocarbon_amount[0]

3.1635710491943385

In [31]:
plotting_df=df_C30

Identifiers = ['LSU','SU','LSL','SL','LPU','PU','LPL','PL','LDU','DU','LDL','DL',]
Labels={'LSU': '','SU': 'Sceptobius Control','LSL': '','SL': 'Sceptobius Treated',
        'LPU': '','PU': 'Platyusa Control','LPL': '','PL': 'Platyusa Treated',
        'LDU': '','DU': 'Dalotia Control','LDL': '','DL': 'Dalotia Treated'}

np.random.seed(666)  

p = bokeh.plotting.figure(plot_width=800,
                          plot_height=600,
                          title='C30',
                          x_range=Identifiers,
                          y_axis_label='ng D62C30',
                          y_axis_type='log')

colors=['#494949','#5F56FF','#494949','#5F56FF','#494949','#C42F2F','#494949','#C42F2F','#494949','#832161','#494949','#832161']

p = box_and_whisker(plotting_df,p,'Identifier','Scaled hydrocarbon amount')

for index, row in plotting_df[['WellNumber','GCMS Date']].drop_duplicates().iterrows():
        
    data = {'identifier': plotting_df['Identifier'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['GCMS Date']==row['GCMS Date'])].values,
            'hydrocarbon amount': plotting_df['Scaled hydrocarbon amount'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['GCMS Date']==row['GCMS Date'])].values}

    source = ColumnDataSource(data=data)
    outlier=np.unique(plotting_df['Outlier'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['GCMS Date']==row['GCMS Date'])])

    offsetVal=(np.random.rand(1)[0]-0.5)*0.5
#     p.line(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
#            'hydrocarbon amount',
#            source=source,
#            color='black',
#            alpha=0.3)

    p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
           'hydrocarbon amount',
           source=source,
           color=bokeh.transform.factor_cmap('identifier',colors,Identifiers),
           alpha=0.6,
           size=7)
    
    if outlier=='AI':
        p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
               'hydrocarbon amount',
               source=source,
               color='lime',
               alpha=1,
               size=7,
               legend='Ant Injured')
    if outlier=='A':
        p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
               'hydrocarbon amount',
               source=source,
               color='purple',
               alpha=1,
               size=7,
               legend='Ant Chomped')
    if outlier=='BI':
        p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
               'hydrocarbon amount',
               source=source,
               color='orange',
               alpha=1,
               size=7,
               legend='Beetle Injured')
    if outlier=='B':
        p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
               'hydrocarbon amount',
               source=source,
               color='pink',
               alpha=1,
               size=7,
               legend='Beetle Chomped')
    if outlier=='Y':
        p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
               'hydrocarbon amount',
               source=source,
               color='cyan',
               alpha=1,
               size=7,
               legend='Ant Crushed or Not Labeled')

p.xgrid.visible = False
p.ygrid.visible = False

# Add custom axis

p.xaxis.formatter = FuncTickFormatter(code="""
var labels = %s;
return labels[tick];
""" %Labels)

bokeh.io.show(p)

In [32]:
df_C24['Scaled Amount']=0
for _,date in enumerate(np.unique(df_C24['GCMS Date'])):
    df_C24.loc[df_C24['GCMS Date']==date,'Scaled Amount'] = preprocessing.MinMaxScaler().fit_transform(df_C24.loc[df_C24['GCMS Date']==date,'Hydrocarbon amount'].values.reshape(-1, 1)).ravel()

df_C30['Scaled Amount']=0
for _,date in enumerate(np.unique(df_C30['GCMS Date'])):
    df_C30.loc[df_C30['GCMS Date']==date,'Scaled Amount'] = preprocessing.MinMaxScaler().fit_transform(df_C30.loc[df_C30['GCMS Date']==date,'Hydrocarbon amount'].values.reshape(-1, 1)).ravel()

/Users/TomNaragon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/TomNaragon/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/TomNaragon/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [33]:
df_C24.head()

,Species,WellNumber,Peak ID,Ret Time,Start Tm,End Tm,m/z,Area,Area Percent,Height,...,Video Date,touching percent,Normalized Area,Hydrocarbon amount,Scaled hydrocarbon amount,Identifier,ant ratio,ant relative amount,scaled ant ratio,Scaled Amount
1,L,1,D50C24,9.498,9.475,9.525,TIC,231950,3.87,222378,...,190919,0.597702,0.041494,1.037348,1.037348,LSL,1.000000,0.000000,1.000000,0.891385
4,S,1,D50C24,9.497,9.475,9.525,TIC,37683,0.62,31671,...,190919,0.597702,0.006242,0.156048,0.261080,SL,0.150430,-0.849570,0.251680,0.129477
7,L,2,D50C24,9.525,9.475,9.525,TIC,1607,0.03,756,...,190919,0.235601,0.000251,0.006282,0.006282,LSU,1.000000,0.000000,1.000000,0.000000
10,S,2,D50C24,9.525,9.475,9.525,TIC,2344,0.04,1432,...,190919,0.235601,0.000388,0.009708,0.041206,SU,1.545441,0.545441,6.559562,0.002962
13,L,3,D50C24,9.525,9.475,9.525,TIC,4472,0.08,2979,...,190919,0.282222,0.000763,0.019070,0.019070,LSU,1.000000,0.000000,1.000000,0.011056


In [34]:
plotting_df=df_C24

Identifiers = ['LSU','SU','LSL','SL','LPU','PU','LPL','PL','LDU','DU','LDL','DL',]
Labels={'LSU': '','SU': 'Sceptobius Control','LSL': '','SL': 'Sceptobius Treated',
        'LPU': '','PU': 'Platyusa Control','LPL': '','PL': 'Platyusa Treated',
        'LDU': '','DU': 'Dalotia Control','LDL': '','DL': 'Dalotia Treated'}

np.random.seed(666)  

p = bokeh.plotting.figure(plot_width=800,
                          plot_height=600,
                          title='C24',
                          x_range=Identifiers,
                          y_axis_label='scaled D50C24')

colors=['#494949','#5F56FF','#494949','#5F56FF','#494949','#C42F2F','#494949','#C42F2F','#494949','#832161','#494949','#832161']

for index, row in plotting_df[['WellNumber','GCMS Date']].drop_duplicates().iterrows():
        
    data = {'identifier': plotting_df['Identifier'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['GCMS Date']==row['GCMS Date'])].values,
            'scaled amount': plotting_df['Scaled Amount'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['GCMS Date']==row['GCMS Date'])].values}

    source = ColumnDataSource(data=data)

    offsetVal=(np.random.rand(1)[0]-0.5)*0.5

    p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
               'scaled amount',
               source=source,
               color=bokeh.transform.factor_cmap('identifier',colors,Identifiers),
               alpha=0.6,
               size=7)
    
    p.line(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
           'scaled amount',
           source=source,
           color='black',
           alpha=0.3)

p.xgrid.visible = False
p.ygrid.visible = False

# Add custom axis

p.xaxis.formatter = FuncTickFormatter(code="""
var labels = %s;
return labels[tick];
""" %Labels)

# p.output_backend = "svg"
# export_svgs(p, filename="DCHC_transfer_C24_minmax_scaled.svg")

bokeh.io.show(p)

In [35]:
plotting_df=df_C30

Identifiers = ['LSU','SU','LSL','SL','LPU','PU','LPL','PL','LDU','DU','LDL','DL',]
Labels={'LSU': '','SU': 'Sceptobius Control','LSL': '','SL': 'Sceptobius Treated',
        'LPU': '','PU': 'Platyusa Control','LPL': '','PL': 'Platyusa Treated',
        'LDU': '','DU': 'Dalotia Control','LDL': '','DL': 'Dalotia Treated'}

np.random.seed(666)  

p = bokeh.plotting.figure(plot_width=800,
                          plot_height=600,
                          title='C30',
                          x_range=Identifiers,
                          y_axis_label='scaled D62C30')

colors=['#494949','#5F56FF','#494949','#5F56FF','#494949','#C42F2F','#494949','#C42F2F','#494949','#832161','#494949','#832161']

for index, row in plotting_df[['WellNumber','GCMS Date']].drop_duplicates().iterrows():
        
    data = {'identifier': plotting_df['Identifier'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['GCMS Date']==row['GCMS Date'])].values,
            'scaled amount': plotting_df['Scaled Amount'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['GCMS Date']==row['GCMS Date'])].values}

    source = ColumnDataSource(data=data)

    offsetVal=(np.random.rand(1)[0]-0.5)*0.5

    p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
               'scaled amount',
               source=source,
               color=bokeh.transform.factor_cmap('identifier',colors,Identifiers),
               alpha=0.6,
               size=7)


p.xgrid.visible = False
p.ygrid.visible = False

# Add custom axis

p.xaxis.formatter = FuncTickFormatter(code="""
var labels = %s;
return labels[tick];
""" %Labels)

# p.output_backend = "svg"
# export_svgs(p, filename="DCHC_transfer_C24_minmax_scaled.svg")

bokeh.io.show(p)

In [36]:
data = plotting_df
x_vals = 'Identifier'
y_vals = 'ant ratio'

groups = data.groupby(x_vals)
q1 = groups.quantile(q=0.25)
q2 = groups.quantile(q=0.5)
q3 = groups.quantile(q=0.75)
iqr = q3 - q1
upper = q3 + 1.5*iqr
lower = q1 - 1.5*iqr

# find the outliers for each category
def outliers(group):
    cat = group.name
    return group[(group[y_vals] > upper.loc[cat][y_vals]) | (group[y_vals] < lower.loc[cat][y_vals])][y_vals]
out = groups.apply(outliers).dropna()

# # prepare outlier data for plotting, we need coordinates for every outlier.
# if not out.empty:
#     outx = []
#     outy = []
#     for keys in out.index:
#         outx.append(keys[0])
#         outy.append(out.loc[keys[0]].loc[keys[1]])

# # if no outliers, shrink lengths of stems to be no longer than the minimums or maximums
# qmin = groups.quantile(q=0.00)
# qmax = groups.quantile(q=1.00)
# upper[y_vals] = [min([x,y]) for (x,y) in zip(list(qmax.loc[:,y_vals]),upper[y_vals])]
# lower[y_vals] = [max([x,y]) for (x,y) in zip(list(qmin.loc[:,y_vals]),lower[y_vals])]

# # stems
# plt.segment(q3.reset_index()[x_vals], upper[y_vals], q3.reset_index()[x_vals], q3[y_vals], line_color="black")
# plt.segment(q3.reset_index()[x_vals], lower[y_vals], q3.reset_index()[x_vals], q1[y_vals], line_color="black")

# #boxes
# plt.vbar(q3.reset_index()[x_vals], 0.7, q2[y_vals], q3[y_vals], fill_color=None, line_color="black")
# plt.vbar(q3.reset_index()[x_vals], 0.7, q1[y_vals], q2[y_vals], fill_color=None, line_color="black")

# # whiskers (almost-0 height rects simpler than segments)
# plt.vbar(q3.reset_index()[x_vals], top=lower[y_vals],bottom=lower[y_vals], width=0.2, line_color="black")
# plt.vbar(q3.reset_index()[x_vals], top=upper[y_vals],bottom=upper[y_vals], width=0.2, line_color="black")

# return plt

KeyError: 'ant ratio'